# Защита персональных данных клиентов

Страховой компании нужен алгоритм шифрования данных клиентов. В этом проекте представлен алгоритм матричного умножения с доказательством его работоспособности: изучено, влияет ли такой алгоритм шифрования на метрику качества модели. Модель предсказывает факт страховых выплат.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from numpy.linalg import inv

In [2]:
insurance_df = pd.read_csv('insurance.csv')

insurance_df.info()
insurance_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Пропусков в данных нет и выглядят они вполне нормально, значит предобработка данных не требуется.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:**

После умножения исходных данных на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:**

Докажем, что $a = Xw = a' = X'w'$ .

$
X' = XP
$

Подставим XP в формулу для весов w и проведем преобразования:

$
w' = ((XP)^T XP)^{-1} (XP)^T y
$

$
w' = (P^T X^T XP)^{-1} P^T X^T y
$

$
w' = (X^T XP)^{-1} (P^T)^{-1} P^T X^T y
$

$
w' = (X^T XP)^{-1} X^T y
$

$
w' = P^{-1} (X^T X)^{-1} X^T y
$

Выражение $(X^T X)^{-1} X^T y$ — это изначальная формула для весов w, можно смело заменить одно другим.

$
w' = P^{-1} w
$

Выражение упрощено максимально. Заменим правую часть равенства $a' = X'w'$ полученными выражениями.

$
X'w' = XP P^{-1} w
$

$
X'w' = Xw
$

Что и требовалось доказать.

## Алгоритм преобразования

**Алгоритм**

1. Получить метрику качества на исходных данных
2. Создать матрицу, состоящую из случайных чисел
3. Проверить полученную матрицу на обратимость
4. Закодировать исходные данные, умножив их на полученную матрицу
5. Получить метрику качества на закодированных данных
6. Сравнить полученные метрики и сделать выводы о проделанной работе

## Проверка алгоритма

In [3]:
def features(df):
    features = df.drop(['Страховые выплаты'], axis=1)
    return features

def target(df):
    target = df['Страховые выплаты']
    return target

features = features(insurance_df)
target = target(insurance_df)

In [4]:
%%time

model = LinearRegression()
model.fit(features, target)
r2 = cross_val_score(model, features, target, cv=5).mean()
print('R2 модели, обученной на исходных данных:', round(r2, 2))

R2 модели, обученной на исходных данных: 0.42
Wall time: 71.8 ms


In [5]:
%%time

matrix = np.random.normal(size=(4, 4))
inv(matrix)
features_new = features.values @ matrix

Wall time: 10.7 ms


In [6]:
%%time

model.fit(features_new, target)
r2_new = cross_val_score(model, features_new, target, cv=5).mean()
print('R2 модели, обученной на зашифрованных данных:', round(r2_new, 2))

R2 модели, обученной на зашифрованных данных: 0.42
Wall time: 6 ms


Проверка алгоритма подтвердила, что шифрование с помощью умножения исходных данных на обратимую матрицу не повлияет на метрику R2. Это означает, что представленный метод шифрования данных может быть полезен в реальных разработках.

Шифрование умножением матриц, по сравнению с более сложными алгоритмами шифрования (например, AES или RSA), требует очень мало процессорного времени, что дает значительную экономию ресурсов.

Однако есть и минусы: главным недостатком представленного метода является увеличение сложности проверки на вырожденность преобразования с ростом количества признаков. В рассмотренном случае мы подобрали подходящую матрицу размером 4х4, но в случае с признаками, исчисляемыми сотнями или даже тысячами, расчеты займут гораздо больше времени, и для такой проблемы необходимо будет найти решение.